In [5]:
from etl.extract.extract_inkipedia import extract_inkipedia
df_weapons = extract_inkipedia()
display(df_weapons)

,Extra_space,Name,ID,Sub,Special,Special_Points,Level,Price,Class,Introduced
0,,Sploosh-o-matic,0,Curling Bomb,Ultra Stamp,180p,9,1,Shooter,Version1.0.0(Drizzle Season 2022)
1,,Neo Sploosh-o-matic,1,Squid Beakon,Killer Wail 5.1,170p,13,1,Shooter,Version3.0.0(Fresh Season 2023)
2,,Splattershot Jr.,10,Splat Bomb,Big Bubbler,180p,1,,Shooter,Version1.0.0(Drizzle Season 2022)
3,,Custom Splattershot Jr.,11,Torpedo,Wave Breaker,190p,6,1,Shooter,Version2.0.0(Chill Season 2022)
4,,Splash-o-matic,20,Burst Bomb,Crab Tank,200p,16,1,Shooter,Version1.0.0(Drizzle Season 2022)
...,...,...,...,...,...,...,...,...,...,...
138,,Order Splatana Replica,8005,Burst Bomb,Zipcaster,210p,1,,Splatana,Version7.0.0(Side Order)
139,,Splatana Wiper,8010,Torpedo,Ultra Stamp,190p,5,1,Splatana,Version1.0.0(Drizzle Season 2022)
140,,Splatana Wiper Deco,8011,Squid Beakon,Tenta Missiles,190p,10,1,Splatana,Version4.0.0(Sizzle Season 2023)
141,,Mint Decavitator,8020,Suction Bomb,Big Bubbler,210p,21,1,Splatana,Version8.0.0(Sizzle Season 2024)


In [12]:
from bs4 import BeautifulSoup
import pandas as pd
from etl.extract.url_request import make_request
import time

MAINS_GALLERY_PATH_1 = (
    'https://splatoonwiki.org/w/index.php?title=Category:Splatoon_3_weapon_icons&fileuntil=S3+Weapon+Main+Order+Splatling+Replica+2D+Current.png#mw-category-media'
)

MAINS_GALLERY_PATH_2 = (
    'https://splatoonwiki.org/w/index.php?title=Category:Splatoon_3_weapon_icons&filefrom=S3+Weapon+Main+Order+Splatling+Replica+2D+Current.png#mw-category-media'
)


# function to extract all main weapon images
# from inkipedia as a dataframe
def extract_mains_images(df_weapons):
    print("\nCreating a dataframe of main weapon images...")
    # first call a function that returns all possible main weapons
    # as a dataframe
    main_weapons_df = find_main_names(df_weapons)
    # find all image urls for main images
    image_urls = []
    # for first page
    image_urls = find_all_main_images(image_urls, MAINS_GALLERY_PATH_1)
    # for second page
    image_urls = find_all_main_images(image_urls, MAINS_GALLERY_PATH_2)
    # add these to the original mains dataframe
    main_weapons_df = create_main_df(main_weapons_df, image_urls)
    print("Done! " + str(main_weapons_df.shape[0]) + "/143 images found")
    # check the length
    if main_weapons_df.shape[0] == 143:
        # return the completed abilities dataframe
        return main_weapons_df
    else:
        raise Exception(
            "Error: Main weapons data frame only has " +
            str(main_weapons_df.shape[0]) +
            " mains, some are missing"
        )


# function to find all possible main weapons
def find_main_names(df_weapons):
    # obtain a series of distinct main weapons
    main_weapons = df_weapons['Name'].unique()
    # create a dataframe of distinct main weapons
    main_weapons_df = pd.DataFrame(main_weapons, columns=['Main Weapon'])
    # return this dataframe of main weapons
    return main_weapons_df


# function to find all main weapon images
def find_all_main_images(image_urls, PATH):
    # make request to the site
    soup = BeautifulSoup(
        make_request(PATH).text, "html.parser"
        )
    # obtain all images
    images = soup.find_all('img')
    # image_urls = []  # to store urls
    # pause
    time.sleep(1)
    # loop through images on the page
    for img in images:
        # pause
        time.sleep(0.05)
        # call a function to add the image url to the list
        image_urls = find_main_image(img, image_urls)
    # return the complete list
    return image_urls


# function to check if the image is a main
# if so add to the list or urls
def find_main_image(img, image_urls):
    src = img.get('src')
    # Check if the 'src' attribute exists and contains 'S3_Weapon_Main'
    # we also don't want the '2D' images
    # also exclude any 'None' images
    if src and 'S3_Weapon_Main' in src and '2D' not in src:
        if 'None' not in src:
            image_urls.append('https:' + src)
    return image_urls


# function to use both the list of images
# and dataframe of weapon names
# to create a complete dataframe of main images
def create_main_df(main_weapons_df, image_urls):
    # first order the mains alphabetically
    main_weapons_df = main_weapons_df.sort_values(by='Main Weapon')
    # reset the index again
    main_weapons_df.reset_index(drop=True, inplace=True)
    # --- Assistance from ChatGPT ---------------------------------------
    # order the specials
    image_urls = sorted(image_urls, key=lambda img: img.split('/')[-1])
    # -------------------------------------------------------------------
    # add a new column called Image URL
    # add the urls here
    main_weapons_df['Image URL'] = image_urls
    # return the complete dataframe
    return main_weapons_df




In [13]:
extract_mains_images(df_weapons)


Creating a dataframe of main weapon images...
Done! 143/143 images found


,Main Weapon,Image URL
0,.52 Gal,https://cdn.wikimg.net/en/splatoonwiki/images/...
1,.52 Gal Deco,https://cdn.wikimg.net/en/splatoonwiki/images/...
2,.96 Gal,https://cdn.wikimg.net/en/splatoonwiki/images/...
3,.96 Gal Deco,https://cdn.wikimg.net/en/splatoonwiki/images/...
4,Aerospray MG,https://cdn.wikimg.net/en/splatoonwiki/images/...
...,...,...
138,Undercover Sorella Brella,https://cdn.wikimg.net/en/splatoonwiki/images/...
139,Wellstring V,https://cdn.wikimg.net/en/splatoonwiki/images/...
140,Z+F Splat Charger,https://cdn.wikimg.net/en/splatoonwiki/images/...
141,Z+F Splatterscope,https://cdn.wikimg.net/en/splatoonwiki/images/...
